In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [5]:
import os
import pandas as pd

## Create All Steps with OpenAI Models

In [6]:
# Gather openai api key
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
# Create the dataset
df = pd.read_csv("/Users/KT71KC/Trainings_and_trials/generative_ai/movies.csv")
df.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,76600,Avatar: The Way of Water,Science Fiction-Adventure-Action,en,Set more than a decade after the events of the first film learn the story of...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,192.0,Released,Return to Pandora.,7.751,6748.0,Sam Worthington-Zoe Saldaña-Sigourney Weaver-Stephen Lang-Kate Winslet-Cliff...,loss of loved one-dying and death-alien life-form-resurrection-sequel-dysfun...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874764-613200-315162-965839-10138...
1,502356,The Super Mario Bros. Movie,Animation-Adventure-Family-Fantasy-Comedy,en,While working underground to fix a water main Brooklyn plumbers—and brothers...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,92.0,Released,NaN,7.556,332.0,Chris Pratt-Anya Taylor-Joy-Charlie Day-Jack Black-Keegan-Michael Key-Seth R...,video game-plumber-magic mushroom-based on video game-aftercreditsstinger-du...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN
2,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's pare...,4704.903,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.732379e+08,125.0,Released,Witness the beginning of a new dynasty.,6.448,1547.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kathryn Newton-Michelle Pfeiffer-...,hero-ant-sequel-superhero-based on comic-family-superhero team-aftercreditss...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,965839-734048-267805-1035806-823999-842942-772515-1058949-1077280-677179-758...
3,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed has been thriving in both his...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Productions-Outlier Society Produ...,2023-03-01,75000000.0,2.690000e+08,116.0,Released,You can't run from your past.,7.262,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majors-Wood Harris-Phylicia Rashād...,philadelphia pennsylvania-husband wife relationship-deaf-sports-sequel-orpha...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-1077280-1058949-772515-937278-640...
4,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,5.200000e+07,100.0,Released,Save your family or save humanity. Make the choice.,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kristen Cui-Nikki Amuka-Bird-Ruper...,based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...


In [8]:
df.shape

(722924, 20)

In [9]:
df['release_date'] = pd.to_datetime(df['release_date'])
df.dtypes

id                               int64
title                           object
genres                          object
original_language               object
overview                        object
popularity                     float64
production_companies            object
release_date            datetime64[ns]
budget                         float64
revenue                        float64
runtime                        float64
status                          object
tagline                         object
vote_average                   float64
vote_count                     float64
credits                         object
keywords                        object
poster_path                     object
backdrop_path                   object
recommendations                 object
dtype: object

In [10]:
# I filtered the momvies after Jan 2022. Because, end date of ChatGPT is Sep 2021. Also, I only picked movies Released after that day.
df_filtered = df[df['release_date']>='2022-01-01']
df_filtered = df_filtered[df_filtered['status']=='Released']
df_filtered.shape

(12573, 20)

In [11]:
df_filtered.release_date.min()

Timestamp('2022-01-01 00:00:00')

In [12]:
# Some preprocessing
df_filtered['credits'] = df_filtered['credits'].str.replace("-",", ")
df_filtered['genres'] = df_filtered['genres'].str.replace("-",", ")
df_filtered.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,76600,Avatar: The Way of Water,"Science Fiction, Adventure, Action",en,Set more than a decade after the events of the first film learn the story of...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,192.0,Released,Return to Pandora.,7.751,6748.0,"Sam Worthington, Zoe Saldaña, Sigourney Weaver, Stephen Lang, Kate Winslet, ...",loss of loved one-dying and death-alien life-form-resurrection-sequel-dysfun...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874764-613200-315162-965839-10138...
1,502356,The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",en,While working underground to fix a water main Brooklyn plumbers—and brothers...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,92.0,Released,NaN,7.556,332.0,"Chris Pratt, Anya Taylor, Joy, Charlie Day, Jack Black, Keegan, Michael Key,...",video game-plumber-magic mushroom-based on video game-aftercreditsstinger-du...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN
2,640146,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Science Fiction",en,Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's pare...,4704.903,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.732379e+08,125.0,Released,Witness the beginning of a new dynasty.,6.448,1547.0,"Paul Rudd, Evangeline Lilly, Jonathan Majors, Kathryn Newton, Michelle Pfeif...",hero-ant-sequel-superhero-based on comic-family-superhero team-aftercreditss...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,965839-734048-267805-1035806-823999-842942-772515-1058949-1077280-677179-758...
3,677179,Creed III,"Drama, Action",en,After dominating the boxing world Adonis Creed has been thriving in both his...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Productions-Outlier Society Produ...,2023-03-01,75000000.0,2.690000e+08,116.0,Released,You can't run from your past.,7.262,1129.0,"Michael B. Jordan, Tessa Thompson, Jonathan Majors, Wood Harris, Phylicia Ra...",philadelphia pennsylvania-husband wife relationship-deaf-sports-sequel-orpha...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-1077280-1058949-772515-937278-640...
4,631842,Knock at the Cabin,"Horror, Mystery, Thriller",en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,5.200000e+07,100.0,Released,Save your family or save humanity. Make the choice.,6.457,888.0,"Dave Bautista, Jonathan Groff, Ben Aldridge, Kristen Cui, Nikki Amuka, Bird,...",based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...


In [13]:
df_filtered.loc[2,'overview']

"Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's parents Janet van Dyne and Hank Pym and Scott's daughter Cassie Lang find themselves exploring the Quantum Realm interacting with strange new creatures and embarking on an adventure that will push them beyond the limits of what they thought possible."

In [14]:
# Create a context for the prompt from the dataset
df_filtered['context'] = df_filtered.apply(lambda row: f"The title of the movie is {row['title']}. The genre of this movie is {row['genres']}. This movie is released on {row['release_date']}. The budget of the movie is ${row['budget']/1000000} million and total revenue of the movie is ${row['revenue']/1000000} million. The plot of the movie is {row['overview']}. Average vote of the movie is {row['vote_average']} out of 10 in {row['vote_count']} votes.", axis =1) 

In [15]:
df_filtered.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations,context
0,76600,Avatar: The Way of Water,"Science Fiction, Adventure, Action",en,Set more than a decade after the events of the first film learn the story of...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,...,Released,Return to Pandora.,7.751,6748.0,"Sam Worthington, Zoe Saldaña, Sigourney Weaver, Stephen Lang, Kate Winslet, ...",loss of loved one-dying and death-alien life-form-resurrection-sequel-dysfun...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874764-613200-315162-965839-10138...,The title of the movie is Avatar: The Way of Water. The genre of this movie ...
1,502356,The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",en,While working underground to fix a water main Brooklyn plumbers—and brothers...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,...,Released,NaN,7.556,332.0,"Chris Pratt, Anya Taylor, Joy, Charlie Day, Jack Black, Keegan, Michael Key,...",video game-plumber-magic mushroom-based on video game-aftercreditsstinger-du...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN,The title of the movie is The Super Mario Bros. Movie. The genre of this mov...
2,640146,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Science Fiction",en,Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's pare...,4704.903,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.732379e+08,...,Released,Witness the beginning of a new dynasty.,6.448,1547.0,"Paul Rudd, Evangeline Lilly, Jonathan Majors, Kathryn Newton, Michelle Pfeif...",hero-ant-sequel-superhero-based on comic-family-superhero team-aftercreditss...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,965839-734048-267805-1035806-823999-842942-772515-1058949-1077280-677179-758...,The title of the movie is Ant-Man and the Wasp: Quantumania. The genre of th...
3,677179,Creed III,"Drama, Action",en,After dominating the boxing world Adonis Creed has been thriving in both his...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Productions-Outlier Society Produ...,2023-03-01,75000000.0,2.690000e+08,...,Released,You can't run from your past.,7.262,1129.0,"Michael B. Jordan, Tessa Thompson, Jonathan Majors, Wood Harris, Phylicia Ra...",philadelphia pennsylvania-husband wife relationship-deaf-sports-sequel-orpha...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-1077280-1058949-772515-937278-640...,"The title of the movie is Creed III. The genre of this movie is Drama, Actio..."
4,631842,Knock at the Cabin,"Horror, Mystery, Thriller",en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,5.200000e+07,...,Released,Save your family or save humanity. Make the choice.,6.457,888.0,"Dave Bautista, Jonathan Groff, Ben Aldridge, Kristen Cui, Nikki Amuka, Bird,...",based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...,The title of the movie is Knock at the Cabin. The genre of this movie is Hor...


In [16]:
df_filtered.loc[0, 'context']

'The title of the movie is Avatar: The Way of Water. The genre of this movie is Science Fiction, Adventure, Action. This movie is released on 2022-12-14 00:00:00. The budget of the movie is $350.0 million and total revenue of the movie is $2312.335665 million. The plot of the movie is Set more than a decade after the events of the first film learn the story of the Sully family (Jake Neytiri and their kids) the trouble that follows them the lengths they go to keep each other safe the battles they fight to stay alive and the tragedies they endure.. Average vote of the movie is 7.751 out of 10 in 6748.0 votes.'

# Create the DocumentStore
### For ElasticSearchDocumentStore, these commands should be run in the command line.
docker pull docker.elastic.co/elasticsearch/elasticsearch:7.9.2

docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.9.2

Note: Docker shoulf be running


In [17]:
from haystack.document_stores import ElasticsearchDocumentStore

In [18]:
document_store = ElasticsearchDocumentStore(embedding_dim=1536, similarity='cosine', return_embedding=True)

In [19]:
document_store

In [22]:
# Write documents into DocumentStore
from haystack import Document

In [23]:
# Put the information inside of documents list to embed document store. 
# We can think of this as our documents
titles = list(df_filtered['title'].values)
contexts = list(df_filtered['context'].values)

documents = []

for title, context in zip(titles, contexts):
  documents.append(Document(content=context, meta={"name":title or ""}))

In [24]:
# Delete existing documents inside of document store if there are any
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

In [25]:
import tiktoken

def num_tokens_from_string(string, encoding_name):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [26]:
# Calculate number of tokens 
df_filtered['token_count'] = df_filtered['context'].apply(lambda text: num_tokens_from_string(text,'cl100k_base'))
df_filtered['word_count'] = df_filtered['context'].apply(lambda text: len(text.split()))

In [27]:
print(df_filtered.token_count.sum())
print(df_filtered.word_count.sum())

1764302
1263523


In [28]:
df_filtered.token_count.sum() * 0.0004 / 1000
# I will pay 70 cents for these embeddings

0.7057208

In [23]:
# Create embeddings with OPENAI model
from haystack.nodes.retriever import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    batch_size = 8,
    embedding_model = 'text-embedding-ada-002',
    api_key = openai_api_key,
    max_seq_len= 8191,
    model_format='openai'
)

document_store.update_embeddings(retriever)
# It took around 11 mins with FAISSDocumentStore and with GPU in Google Colab.
# In my laptop, it took around 20 mins to calculate.


In [32]:
document_store.update_embeddings(retriever)
# On top of embedding retrieval for 20 mins, it took another 20 mins to put embeddings to document store.

Calculating embeddings: 100%|██████████| 214/214 [03:39<00:00,  1.03s/it]s/s]
Updating embeddings: 20000 Docs [25:18, 13.17 Docs/s]                        


In [20]:
document_store.get_document_count()

11708

In [21]:
document_store.get_embedding_count()

11708

In [24]:
# Let's just test the retriever and document store
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

search_pipe = DocumentSearchPipeline(retriever=retriever)
result = search_pipe.run(
    query="What is the plot of Avatar The Way of Water?",
    params = {'Retriever': {'top_k': 2}}
)

print_documents(result)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.90it/s]


Query: What is the plot of Avatar The Way of Water?

{   'content': 'The title of the movie is Avatar: The Way of Water. The genre '
               'of this movie is Science Fiction, Adventure, Action. This '
               'movie is released on 2022-12-14 00:00:00. The budget of the '
               'movie is $350.0 million and total revenue of the movie is '
               '$2312.335665 million. The plot of the movie is Set more than a '
               'decade after the events of the first film learn the story of '
               'the Sully family (Jake Neytiri and their kids) the trouble '
               'that follows them the lengths they go to keep each other safe '
               'the battles they fight to stay alive and the tragedies they '
               'endure.. Average vote of the movie is 7.751 out of 10 in '
               '6748.0 votes.',
    'name': 'Avatar: The Way of Water'}

{   'content': 'The title of the movie is The Call of Water. The genre of this '
            

In [48]:
# Create the generator by using PromptNode to use OpenAI ChatCompletion Models
from haystack.nodes import PromptNode

generator = PromptNode(model_name_or_path="gpt-3.5-turbo", api_key=openai_api_key, default_prompt_template='question-answering', max_length=256)

In [49]:
from haystack.pipelines import GenerativeQAPipeline
# Initiate the pipeline
pipeline = GenerativeQAPipeline(generator = generator, retriever = retriever)


In [50]:
query = "What is the plot of the movie Knock at the Cabin and what was the revenue generated from this movie? What is the genre of this movie and accordingly, can you recommend me 3 other  popular movies in the same genre? "

In [51]:
response = pipeline.run(query = query)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


In [52]:
response['answers'][0].answer

'The plot of the movie Knock at the Cabin is that while vacationing at a remote cabin a young girl and her two fathers are taken hostage by four armed strangers who demand that the family make an unthinkable choice to avert the apocalypse. The total revenue generated from this movie is $52.0 million. The genre of this movie is Horror, Mystery, Thriller. Three other popular movies in the same genre are The Conjuring, Get Out, and A Quiet Place.'

In [ ]:
# API call to OpenAI, sometimes gives timeout error. Especially, when I try with gpt-4. 